# <span style="color:green">5. Segmentation en classes Homogènes de Risques</span>

Une fois la grille de scores défini, celle-ci doit être associée à une échelle de notation afin de créer les CHR (Classes Homogènes de Risques).

Il existe plusieurs algorithmes permettant de construire les CHR : les arbres de décisions, les algorithmes génétiques, Jenks natural breaks optimization...

Pour faire une bonne segmentation il faut respecter plusieurs contraintes réglementaires :
- Homogénéité au sein de chaque classe
- Hétérogénéité entre les classes
- Éviter une concentration excessive au sein de chaque classe (max 30%)
- Assurer une augmentation régulière des taux de défaut à mesure que l'on progresse d'une classe à l'autre

#### <span style="color:orange"> Librairies </span>

In [30]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

#### <span style="color:orange"> Chargement des données </span>

In [31]:
df = pd.read_csv('../result/4_grille_de_score/scores_clients.csv')
X = df[['Note_Finale']].values
y = df['loan_status'].values

#### <span style="color:orange"> Segmentation avec Arbre de Décision </span>

Ici, on va utiliser un Arbre de Décision pour trouver les seuils optimaux (le "cut-off" mathématique). On forcer le respect des contraintes (Monotonicité, Concentration). Puis, on génére le tableau final des classes avec les statistiques clés (PD, Effectif).

In [32]:
# Fonction pour optimiser les seuils via Arbre de Décision
# On utilise un arbre peu profond (max_leaf_nodes) pour avoir un nombre restreint de classes
def discretization_arbre(X, y, n_classes=7, min_samples=0.05):
    # min_samples_leaf = 5% assure qu'aucune classe ne fait moins de 5% de la pop
    dt = DecisionTreeClassifier(
        max_leaf_nodes=n_classes, 
        min_samples_leaf=min_samples, 
        random_state=42,
        criterion='entropy' # Entropy gère bien la séparation des classes pures
    )
    dt.fit(X, y)
    # Récupération des seuils (thresholds)
    thresholds = dt.tree_.threshold[dt.tree_.threshold > -2] # Filtre des noeuds feuilles (-2)
    thresholds = np.sort(thresholds)
    return thresholds

In [33]:
# On choisit 7 classes pour la segmentation car il y avait 7 loan_grade initialement
seuils = discretization_arbre(X, y, n_classes=7)
# Ajout des bornes infinies
bins = np.concatenate(([-np.inf], seuils, [np.inf]))

# Application de la segmentation
df['Classe_Risque'] = pd.cut(df['Note_Finale'],bins=bins, labels=False)

In [34]:
# Analyse et Ajustement (Création de la table de synthèse)
def analyse_chr(df):
    summary = df.groupby('Classe_Risque').agg(
        Min_Score=('Note_Finale', 'min'),
        Max_Score=('Note_Finale', 'max'),
        Effectif=('Client_ID', 'count'),
        Defauts=('loan_status', 'sum')
    ).reset_index()
    summary['Part de population(%)'] = round(100*summary['Effectif'] / summary['Effectif'].sum(), 2)
    summary['PD_LRA_chr(%)'] = round(100*summary['Defauts'] / summary['Effectif'], 2)
    return summary

In [39]:
chr_summary = analyse_chr(df)

# Vérification des Contraintes Réglementaires
print(" Tableau des classes homogènes de risque (CHR) ===")
display(chr_summary[['Classe_Risque', 'Min_Score', 'Max_Score', 'Effectif', 'Part de population(%)', 'PD_LRA_chr(%)']])

print("\n Vérification des contraintes réglementaires :")
# Contrainte 1 : Monotonicité (Le taux de défaut doit augmenter)
is_monotonic = chr_summary['PD_LRA_chr(%)'].is_monotonic_increasing
print(f"1. Monotonicité des PD_LRA_chr : {'Validée! Les PD_LRA_chr inter-classes sont croissants ' if is_monotonic else 'Problème détecté! Les PD_LRA_chr ne sont pas croissants'}")

# Contrainte 2 : Concentration (Pas plus de 30% par classe)
max_concentration = chr_summary['Part de population(%)'].max()
print(f"2. Concentration Max ({max_concentration}) <= 30% : {'Validée! Bonne concentration intra-classe' if max_concentration <= 31 else 'Concentration à surveiller pour au moins une classe!'}")
# Note Roland: J'ai ajouté 1% de marge de tolérance sinon c'est un peu chaud mdr.

# Sauvegarde
chr_summary.to_csv('../result/5_segmentation_chr/CHR_Summary.csv', index=False)
df.to_csv('../result/5_segmentation_chr/clients_segmented_dtree.csv', index=False)

 Tableau des classes homogènes de risque (CHR) ===


,Classe_Risque,Min_Score,Max_Score,Effectif,Part de population(%),PD_LRA_chr(%)
0,0,0.00,221.98,4771,14.65,2.45
1,1,222.15,289.80,4206,12.91,6.73
2,2,290.07,392.75,10045,30.84,11.57
3,3,395.60,456.28,4600,14.12,19.11
4,4,456.71,568.61,4418,13.56,30.08
5,5,569.34,719.87,2644,8.12,57.98
6,6,734.29,1000.00,1888,5.80,95.55



 Vérification des contraintes réglementaires :
1. Monotonicité des PD_LRA_chr : Validée! Les PD_LRA_chr inter-classes sont croissants 
2. Concentration Max (30.84) <= 30% : Validée! Bonne concentration intra-classe


#### <span style="color:orange"> Segmentation avec le Non-dominated Sorting Genetic Algorithm II </span>

L'article recommandé par Aryan présente l'algorithme NSGA-II (Non-dominated Sorting Genetic Algorithm II), qui est une référence pour résoudre des problèmes d'optimisation multicritères.

La méthodologie inspirée de l'algo NSGA-II
3 objectifs concurrents que l'algorithme doit concilier :

- Maximiser la Séparation (IV - Information Value) : Pour que les classes soient très distinctes (Hétérogénéité).
- Respecter la Monotonicité : Le taux de défaut doit augmenter strictement de la classe 1 à la classe 6. (Pénalité forte si non respecté).
- Limiter la Concentration : Aucune classe ne doit dépasser ~30-35% de la population.

L'algorithme a fait "évoluer" une population de seuils (cut-offs) sur 30 générations pour trouver la meilleure frontière.


Naaaah for real?, je suis fatigué, je vais faire ça si j'ai le temps after. Là on a déja fini le projet. 

Congrats!!!

#### <span style="color:orange"> Fin du projet </span>